In [ ]:
import os
from pathlib import Path

import syft_rds as sy

import syft_flwr

In [ ]:
# EXAMPLE = "embedded-devices"
EXAMPLE = "quickstart-pytorch"

if EXAMPLE == "embedded-devices":
    SYFTBOX_DATASET_NAME = "fashionmnist"
elif EXAMPLE == "quickstart-pytorch":
    SYFTBOX_DATASET_NAME = "cifar10"

In [ ]:
DS = "khoa@openmined.org"

!just dump-sim-config {DS}

In [ ]:
PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE
SIM_DIR = PROJECT_DIR / ".sim"
CONFIG_DIR = SIM_DIR / ".config"

DO1 = "yash@openmined.org"
DO2 = "rasswanth@openmined.org"
DS_CONFIG = f"{CONFIG_DIR}/{DS}.json"

In [ ]:
do_client_1 = sy.init_session(host=DO1, syftbox_client_config_path=DS_CONFIG)
assert do_client_1.host == DO1
assert do_client_1.email == DS
assert not do_client_1.is_admin

do_client_2 = sy.init_session(host=DO2, syftbox_client_config_path=DS_CONFIG)
assert do_client_2.host == DO2
assert do_client_2.email == DS
assert not do_client_2.is_admin

In [ ]:
datasites = [do_client_1, do_client_2]

dataset1 = do_client_1.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset2.describe()

## Bootstrap the `flwr` project and run the server side

In [ ]:
try:
    syft_flwr.bootstrap(FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2])
    print("Project bootstrapped successfully")
except Exception as e:
    print(e)

In [ ]:
!just run-sim-server {str(FLWR_PROJECT_PATH)} {DS}

## TODO - Run simulation (locally on DS's machine)


In [ ]:
syft_flwr.run_simulation(
    path=FLWR_PROJECT_PATH,
    mock_paths=[
        do_client.datasets.get(name=SYFTBOX_DATASET_NAME).mock_path
        for do_client in datasites
    ],
    num_datasites=len(datasites),
)

## TODO - Submit Jobs

In [ ]:
# Job submission
for client in datasites:
    client.jobs.submit(
        name="MedMNIST FL Experiment",
        description="MedMNIST Federated Learning Experiment",
        user_code_path=FLWR_PROJECT_PATH,
        # runtime="./dockerfile", # path to a local dockerfile
        tags=["federated", "learning", "medical", "mnist"],
    )

syft_flwr.view_server_logs(name=FLWR_PROJECT_PATH.name)